In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json
import time
import pandas as pd
import datetime as dt
from selenium import webdriver
import re

In [2]:
url = 'https://phongvu.vn/laptop-theo-thuong-hieu-13084.cat?pv_source=homepage&pv_medium=de-megamenu-text&sort=bestPrice&order=ASC&page=1'
browser = webdriver.Chrome(executable_path='./chromedriver.exe') 
browser.get('https://phongvu.vn/laptop-theo-thuong-hieu-13084.cat?pv_source=homepage&pv_medium=de-megamenu-text&sort=bestPrice&order=ASC&page=1')
html_text = browser.page_source
browser.quit()
tree = BeautifulSoup(html_text, 'html.parser')

In [3]:
laps = tree.find_all('a', {'class' : 'css-1rhapru'})

In [4]:
fileout = open('laps_info.tsv', 'w', encoding = 'utf-8')
last_page = tree.select('ul.css-19x39nn li:nth-of-type(10)')[0]['data-content-target']
count = 0
data_dicts = []
for i in range(int(last_page)):
    count+=1
    print(count)
    if count > 1:
        browser = webdriver.Chrome(executable_path='./chromedriver.exe')
        browser.get('https://phongvu.vn/laptop-theo-thuong-hieu-13084.cat?pv_source=homepage&pv_medium=de-megamenu-text&sort=bestPrice&order=ASC&page='+str(count))
        html_text = browser.page_source #requests each page one by one till last page  
        browser.quit()
        tree = BeautifulSoup(html_text, 'html.parser')
    laps = tree.find_all('a', {'class' : 'css-1rhapru'})
    cnt = 1
    for lap in laps:
        row_dict = dict()
        print("cnt: ", cnt)
        cnt += 1
        # Vào link lấy thông tin chi tiết của laptop
        inner_url = lap.get('href')
        inner_url = 'https://phongvu.vn/' + str(inner_url)
        row_dict['URL'] = inner_url
        print("inner_url: ", inner_url)
        browser = webdriver.Chrome(executable_path='./chromedriver.exe')
        browser.get(inner_url)
        lap_html_text = browser.page_source
        browser.quit()
        lap_tree = BeautifulSoup(lap_html_text, 'html.parser')
        lap_name = lap_tree.find('div', {'class' : 'css-1jpdzyd'}).get_text(strip = True) # Get the laptop name
        row_dict['Tên laptop'] = lap_name
        print("name: ", lap_name)
        lap_price = lap_tree.find('span', {'class' : 'css-3725in'}).get_text(strip = True)
        row_dict['Giá  laptop'] = lap_price
        print("price: ", lap_price)
        tech_info = lap_tree.find('div', {'class' : 'css-9zicy3'})
        description = tech_info.find_all('span', {'class' : 'css-6z2lgz'})
        header_list = []
        for i in range (len(description)):
            header_list.append(description[i].text)
            
        for i in range (len(header_list)):
            temp = lap_tree.find('span', text = header_list[i]).find_next('span').find_next('div').get_text(strip = True)
            col_name = header_list[i]
            col_val = temp
            row_dict[col_name] = col_val
        data_dicts.append(row_dict)
        
df = pd.DataFrame(data_dicts)
df.to_csv('lap_info.csv', index=False)     
        

        

1
cnt:  1
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-x407ma-bv043t-n4000-vang-dong-s1805439.html
name:  Laptop ASUS VivoBook X407MA-BV043T (14" HD/N4000/4GB/1TB HDD/UHD 600/Win10/1.5 kg)
price:  6.390.000đ
cnt:  2
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-e210ma-gj083t-celeron-n4020-den-s201000020.html
name:  Laptop ASUS Vivobook E210MA GJ083T ( 11.6" HD/Intel Celeron N4020/4GB/128GB SSD/Windows 10 Home SL 64-bit/1kg)
price:  5.990.000đ
cnt:  3
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-acer-aspire-3-a31534c38y-nxhe3sv00g-celeron-n4020-den-s200601143.html
name:  Laptop ACER Aspire 3 A315-34-C38Y NX.HE3SV.00G ( 15.6" HD/Intel Celeron N4020/4GB/256GB SSD/Windows 10 Home 64-bit/1.7kg)
price:  6.990.000đ
cnt:  4
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-x509ma-br270t-n4020-bac-s201200081.html
name:  Laptop ASUS Vivobook X509MA BR270T ( 15.6" HD/Intel Celeron N4020/4GB/256GB SSD/Windows 10 Home SL 64-

name:  Laptop Acer Aspire 5 A515-53-3153 (NX.H6BSV.005) (15.6" FHD/i3-8145U/4GB/1TB HDD/Win10/2.1 kg)
price:  11.990.000đ
cnt:  25
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-pro-p1440ua-fq0183-i3-8130u-xam-s19040052.html
name:  Laptop ASUS PRO P1440UA-FQ0183 (14" HD/i3-8130U/4GB/500GB HDD/UHD 620/Endless/1.7 kg)
price:  11.990.000đ
cnt:  26
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-x509ja-ej543t-i3-1005g1u-bac-s200700378.html
name:  Laptop ASUS Vivobook X509JA-EJ543T EJ543T ( 15.6" Full HD/Intel Core i3-1005G1/4GB/512GB SSD/Windows 10 Home SL 64-bit/1.7kg)
price:  11.190.000đ
cnt:  27
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-a512fa-ej117t-i3-8145u-bac-s19040955.html
name:  Laptop ASUS VivoBook 15 A512FA-EJ117T (15.6" FHD/i3-8145U/4GB/1TB HDD/UHD 620/Win10/1.7 kg)
price:  12.290.000đ
cnt:  28
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-a412fa-ek377t-i3-8145u-bac-s19070191.html
name:  Laptop A

name:  Laptop Dell Inspiron 15 3505 Y1N1T1 ( 15.6" Full HD/AMD Ryzen 3 3250U/8GB/256GB SSD/Windows 10 Home SL 64-bit/1.8kg)
price:  12.190.000đ
cnt:  18
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-dell-inspiron-14-3481-3481-030cx1-i3-7020u-bac-s19060172.html
name:  Laptop Dell Inspiron 3481-030CX1 (14" HD/i3-7020U/4GB/1TB HDD/HD 620/Win10/1.7 kg)
price:  11.780.000đ
cnt:  19
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-acer-aspire-5-a514533821-nxhussv001-i31005g1-bac-s200400304.html
name:  Laptop Acer Aspire 5 A514-53-3821 (NX.HUSSV.001) (14" FHD/i3-1005G1/4GB/256GB SSD/Intel UHD/Win10/1.5kg)
price:  13.190.000đ
cnt:  20
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-lenovo-ideapad-330-15ikbr-81de010cvn-i3-7020u-den-s1808260.html
name:  Laptop Lenovo Ideapad 330-15IKBR (15IKBR 81DE010CVN) (15.6" HD/i3-7020U/4GB/2TB HDD/Radeon 530/Win10/2.2 kg)
price:  11.990.000đ
cnt:  21
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-acer-aspire-a514-51-58z

name:  Laptop HP ProBook 440 G5 (2ZD34PA) (14" HD/i3-7100U/4GB/500GB HDD/HD 620/Free DOS/1.6 kg)
price:  12.590.000đ
cnt:  11
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-lenovo-ideapad-330-15ikb-81de01jsvn-i5-8250u-den-s1809644.html
name:  Laptop Lenovo 330-15IKB (81DE01JSVN) (15.6" HD/i5-8250U/4GB/500GB HDD/Radeon R7 M530/Free DOS/1.9 kg)
price:  12.590.000đ
cnt:  12
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-x407ufbv056t-i58250u-vang-s190800393.html
name:  Laptop ASUS VivoBook X407UF-BV056T (14" HD/i5-8250U/4GB/1TB HDD/MX130/Win10/1.6 kg)
price:  13.390.000đ
cnt:  13
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-a412fa-ek1188t-i3-10110u-bac-s200801174.html
name:  Laptop ASUS Vivobook A412FA EK1188T ( 14" Full HD/Intel Core i3-10110U/4GB/256GB SSD/Windows 10 Home 64-bit/1.3kg)
price:  12.590.000đ
cnt:  14
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-hp-340s-g7-224l1pa-i3-1005g1-xam-s200900867.html
name:  Laptop HP 34

name:  Laptop HP Pavilion 15-cs2000TU (6KX21PA) (15.6" FHD/i3-8145U/4GB/1TB HDD/UHD 620/Win10/1.8 kg)
price:  13.290.000đ
cnt:  5
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-hp-pavilion-14-ce2034tu-6yz17pa-i3-8145u-bac-s19060191.html
name:  Laptop HP Pavilion 14-ce2034TU (6YZ17PA) (14" FHD/i3-8145U/4GB/1TB HDD/UHD 620/Win10/1.6 kg)
price:  13.290.000đ
cnt:  6
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-hp-pavilion-15cs2031tu-6yz03pa-i38145u-vang-s190800051.html
name:  Laptop HP Pavilion 15-cs2031TU (6YZ03PA) (15" FHD/i3-8145U/4GB/1TB HDD/UHD 620/Win10/1.8 kg)
price:  13.290.000đ
cnt:  7
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-dell-vostro-15-3590-3590v5i3505w-i310110u-den-s200201232.html
name:  Laptop Dell Vostro 15 3590 3590-V5I3505W ( 15.6" Full HD/Intel Core i3-10110U/4GB/1TB HDD/Windows 10 Home 64-bit/2.2kg)
price:  13.290.000đ
cnt:  8
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-x509faej203t-i58265u-bac-s190800609

...

name:  Laptop HP Pavilion 15-cs2056TX (6YZ11PA) (15" FHD/i5-8265U/4GB/1TB HDD/GeForce MX130/Win10/1.9 kg)
price:  16.490.000đ
cnt:  17
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-hp-pavilion-15-cs2057tx-6yz20pa-i5-8265u-xam-s19060294.html
name:  Laptop HP Pavilion 15-cs2057TX (6YZ20PA) (15" FHD/i5-8265U/4GB/1TB HDD/GeForce MX130/Win10/1.8 kg)
price:  16.490.000đ
cnt:  18
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-hp-348-g7-9ph00pa-i510210u-bac-s200201204.html
name:  Laptop HP 348 G7 (9PH00PA) (14" FHD/i5-10210U/8GB/256GB SSD/Intel UHD/Win10/1.4kg)
price:  15.390.000đ
cnt:  19
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-m413ia-ek338t-amd-ryzen-5-4500u-bac-s200801033.html
name:  Laptop ASUS Vivobook M413IA EK338T ( 14" Full HD/AMD Ryzen 5 4500U/8GB/512GB SSD/Windows 10 Home 64-bit/1.5kg)
price:  15.490.000đ
cnt:  20
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-x509jpej013t-i51035g1-bac-s200500678.html
name:  

name:  Laptop Dell Inspiron 15 5593-N5I5513W (15.6" FHD/i5-1035G1/8GB/256GB SSD/GeForce MX230/Win10/1.9kg)
price:  17.990.000đ
cnt:  9
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-acer-aspire-7-a71541gr8kq-nhq8dsv001-amd-ryzen-5-3550h-den-s200601142.html
name:  Laptop Acer Aspire 7 A715-41G-R8KQ - Đen
price:  17.990.000đ
cnt:  10
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-a512flej166t-i78565u-bac-s190800425.html
name:  Laptop ASUS VivoBook 15 A512FL-EJ166T (15" FHD/i7-8565U/8GB/512GB SSD/MX250/Win10/1.7 kg)
price:  19.990.000đ
cnt:  11
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-hp-probook-450-g79gq40pa-i510210u-bac-s200201189.html
name:  Laptop HP ProBook 450 G7-9GQ40PA (15.6" FHD/i5-10210U/8GB/256GB SSD/Intel UHD/Win10/1.9kg)
price:  18.990.000đ
cnt:  12
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-s433faeb053t-i510210u-den-s200600698.html
name:  Laptop ASUS Vivobook S433FA-EB053T 90NB0Q04-M00820 ( 14" Ful

name:  Laptop HP Pavilion 15-cb540TX (4BN72PA) (15.6" FHD/i5-7300HQ/8GB/1TB HDD/GTX 1050/Win10/2.3 kg)
price:  22.190.000đ
19
cnt:  1
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-zenbook-ux333fa-a4184t-i5-8265u-do-burgendy-s19030194.html
name:  Laptop ASUS ZenBook 13 UX333FA-A4184T (13.3" FHD/i5-8265U/8GB/512GB SSD/UHD 620/Win10/1.2 kg)
price:  22.490.000đ
cnt:  2
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-zenbook-ux333fa-a4118t-i5-8265u-xanh-s19060179.html
name:  Laptop ASUS ZenBook 13 UX333FA-A4118T (13" FHD/i5-8265U/8GB/512GB SSD/UHD 620/Win10/1.2 kg)
price:  22.490.000đ
cnt:  3
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-acer-nitro-5-an51554595d-nhq59sv025-i59300h-den-s190800380.html
name:  Laptop Acer Nitro 5 AN515-54-595D (NH.Q59SV.025) (15" FHD/i5-9300H/8GB/512GB SSD/GTX 1650/Win10/2.3 kg)
price:  21.990.000đ
cnt:  4
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-acer-nitro-5-an5155476rk-nhq59sv023-i79750h-den-s190800388

name:  Laptop Dell Inspiron 7591-N5I5591W (15.6" FHD/i5-9300H/8GB/256GB SSD/GeForce GTX 1050/Win10/2kg)
price:  25.490.000đ
cnt:  33
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-hp-probook-440-g8-2z6j6pa-i7-1165g7-bac-s201200513.html
name:  Laptop HP ProBook 440 G8 2Z6J6PA ( 14" Full HD/Intel Core i7-1165G7/16GB/512GB SSD/Windows 10 Home SL 64-bit/1.4kg)
price:  25.490.000đ
cnt:  34
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-dell-inspiron-5482-c2cpx1-i7-8565u-bac-s18110312.html
name:  Laptop Dell Inspiron 5482-C2CPX1 (14" FHD/i7-8565U/8GB/256GB SSD/MX130/Win10/2 kg)
price:  27.990.000đ
cnt:  35
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-zenbook-ux425ea-bm113t-i7-1165g7-xam-s201100289.html
name:  Laptop ASUS Zenbook UX425EA BM113T ( 14" Full HD/Intel Core i7-1165G7/16GB/512GB SSD/Windows 10 Home 64-bit/1.2kg)
price:  28.990.000đ
cnt:  36
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-hp-envy-x360-13-ay0069au-171n3pa-amd-ryzen-7-4700

TypeError: __init__() got an unexpected keyword argument 'ignore_index'

In [11]:
data_dicts
print(len(data_dicts))
df = pd.DataFrame(data_dicts)

752


In [16]:
url = 'https://phongvu.vn/laptop-theo-thuong-hieu-13084.cat?pv_source=homepage&pv_medium=de-megamenu-text&sort=bestPrice&order=ASC&page=1'
browser = webdriver.Chrome(executable_path='./chromedriver.exe') 
browser.get('https://phongvu.vn/laptop-theo-thuong-hieu-13084.cat?pv_source=homepage&pv_medium=de-megamenu-text&sort=bestPrice&order=ASC&page=1')
html_text = browser.page_source
browser.quit()
tree = BeautifulSoup(html_text, 'html.parser')

data_dictsl = []
count = 0
for i in range(int(last_page)):
    count+=1
    print(count)
    if count > 1:
        browser = webdriver.Chrome(executable_path='./chromedriver.exe')
        browser.get('https://phongvu.vn/laptop-theo-thuong-hieu-13084.cat?pv_source=homepage&pv_medium=de-megamenu-text&sort=bestPrice&order=ASC&page='+str(count))
        html_text = browser.page_source #requests each page one by one till last page  
        browser.quit()
        tree = BeautifulSoup(html_text, 'html.parser')
    laps = tree.find_all('a', {'class' : 'css-1rhapru'})
    cnt = 1
    for lap in laps:
        row_dict = dict()
        print("cnt: ", cnt)
        cnt += 1
        # Vào link lấy thông tin chi tiết của laptop
        inner_url = lap.get('href')
        inner_url = 'https://phongvu.vn/' + str(inner_url)
        row_dict['URL'] = inner_url
        print("inner_url: ", inner_url)
        browser = webdriver.Chrome(executable_path='./chromedriver.exe')
        browser.get(inner_url)
        lap_html_text = browser.page_source
        browser.quit()
        lap_tree = BeautifulSoup(lap_html_text, 'html.parser')
        lap_name = lap_tree.find('div', {'class' : 'css-1jpdzyd'}).get_text(strip = True) # Get the laptop name
        row_dict['Tên laptop'] = lap_name
        print("name: ", lap_name)
        lap_price = lap_tree.find('span', {'class' : 'css-3725in'}).get_text(strip = True)
        row_dict['Giá  laptop'] = lap_price
        print("price: ", lap_price)
        tech_info = lap_tree.find('div', {'class' : 'css-9zicy3'})
        
        description = tech_info.find_all('span', {'class' : 'css-6z2lgz'})
        header_list = []
        for i in range (len(description)):
            header_list.append(description[i].text)
            
        for i in range (len(header_list)):
            temp = lap_tree.find('span', text = header_list[i]).find_next('span').find_next('div').get_text(strip = True)
            col_name = header_list[i]
            col_val = temp
            row_dict[col_name] = col_val
        data_dictsl.append(row_dict)
    if count == 1:
        break

1
cnt:  1
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-x407ma-bv043t-n4000-vang-dong-s1805439.html
name:  Laptop ASUS VivoBook X407MA-BV043T (14" HD/N4000/4GB/1TB HDD/UHD 600/Win10/1.5 kg)
price:  6.390.000đ
cnt:  2
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-e210ma-gj083t-celeron-n4020-den-s201000020.html
name:  Laptop ASUS Vivobook E210MA GJ083T ( 11.6" HD/Intel Celeron N4020/4GB/128GB SSD/Windows 10 Home SL 64-bit/1kg)
price:  5.990.000đ
cnt:  3
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-acer-aspire-3-a31534c38y-nxhe3sv00g-celeron-n4020-den-s200601143.html
name:  Laptop ACER Aspire 3 A315-34-C38Y NX.HE3SV.00G ( 15.6" HD/Intel Celeron N4020/4GB/256GB SSD/Windows 10 Home 64-bit/1.7kg)
price:  6.990.000đ
cnt:  4
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-asus-vivobook-x509ma-br270t-n4020-bac-s201200081.html
name:  Laptop ASUS Vivobook X509MA BR270T ( 15.6" HD/Intel Celeron N4020/4GB/256GB SSD/Windows 10 Home SL 64-

name:  Laptop ASUS 15 X509FA-EJ199T (15.6" FHD/i3-8145U/4GB/1TB HDD/Intel UHD/Win10/1.8kg)
price:  10.590.000đ
cnt:  35
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-lenovo-ideapad-3-15ada05-81w100guvn-amd-ryzen-3-3250u-xam-s201000603.html
name:  Laptop Lenovo Ideapad 3 15ADA05 81W100GUVN ( 15.6" Full HD/AMD Ryzen 3 3250U/4GB/256GB SSD/Windows 10 Home 64-bit/1.8kg)
price:  9.790.000đ
cnt:  36
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-acer-aspire-3-a31542r8px-nxhf9sv00a-amd-ryzen-3-3200u-den-s191005043.html
name:  Laptop Acer Aspire 3 A315-42-R8PX (NX.HF9SV.00A) (15" FHD/R3-3200U/8GB/256GB SSD/Radeon Vega 3/Win10/1.7 kg)
price:  10.990.000đ
cnt:  37
inner_url:  https://phongvu.vn//may-tinh-xach-tay-laptop-dell-vostro-14-3405-3405-v4r33250u501w-amd-ryzen-3-3250u-den-s201100385.html
name:  Laptop Dell Vostro 14 3405 V4R33250U501W ( 14" HD/AMD Ryzen 3 3250U/4GB/1TB HDD/Windows 10 Home SL 64-bit/1.7kg)
price:  9.900.000đ
cnt:  38
inner_url:  https://phongvu.vn//may

In [1]:
my_finallist = [i for j, i in enumerate(data_dicts) if i not in data_dicts[:j]] 
print(len(my_finallist))
df = pd.DataFrame(my_finallist)
df.to_csv('lap_info.csv', index=False)   

NameError: name 'data_dicts' is not defined